# News Category Data Review

Load the news category data into the memory. It takes a few minutes and please be pacient. 

In [1]:
import tensorflow as tf
import pandas as pd
import Run_Prediction as rp
import NewsCategoryData as ncd
from NewsCategoryData import NewsCategory
from NewsCategoryData import LABEL_DIC
from NewsCategoryData import LABEL_LIST


BATCH_SIZE = 16
data = NewsCategory(batch_size=BATCH_SIZE)
print("News category data load completed. Total %d recorders."%len(data.data))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wangdi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


News category data load completed. Total 200847 recorders.


## Run below codes to print all the labels and label index

In [2]:
od = ncd.print_all_class()
od

,0,1,2,3,4,5,6,7,8,9
0,CRIME,0,ENTERTAINMENT,1,WORLD NEWS,2,IMPACT,3,POLITICS,4
1,BLACK VOICES,6,WOMEN,7,COMEDY,8,QUEER VOICES,9,SPORTS,10
2,TRAVEL,12,MEDIA,13,TECH,14,RELIGION,15,SCIENCE,16
3,EDUCATION,18,COLLEGE,19,PARENTS,20,ARTS & CULTURE,21,STYLE,22
4,TASTE,24,HEALTHY LIVING,25,THE WORLDPOST,26,GOOD NEWS,27,WORLDPOST,28
5,ARTS,30,WELLNESS,31,PARENTING,32,HOME & LIVING,33,STYLE & BEAUTY,34


## Run below code to show 10 news title at specific label
Input the index of label you want to review.

In [3]:
MAX_SHOW = 10

def is_int(s):
    try:
        int(s)
        return True
    except ValueError:
        pass

index = 0

while True:
    input_value = input("Please input a label index(0-40):") 
    if is_int(input_value):
        index = int(input_value)
        if index <=40 or index >=0:
            break
        
icount = 0

for i in range(data.max_recorder):
    if data.label[i] == index:
        news_title = ""
        for word in data.data[i]:
            news_title += word + " "
        print(LABEL_LIST[data.label[i]]," ", news_title)
        icount += 1
        if icount > MAX_SHOW:
            break
   

Please input a label index(0-40): 3


IMPACT   With Its Way Of Life At Risk , This Remote Oyster - Growing Region Called In Robots 
IMPACT   Monsanto And Bayer Are Set To Merge . Heres Why You Should Care . 
IMPACT   Youre Going To Use That Self - Checkout Machine Whether You Like It Or Not 
IMPACT   Machines Dont Always Steal Our Jobs . Increasingly , They Rob Our Work Of Humanity . 
IMPACT   She Started A Suicide Prevention Site At Age 15 . Its Still Going Strong . 
IMPACT   This CEO Took A Pay Cut To Give Employees $ 70 , 000 A Year . Now He ’ s Battling Amazon . 
IMPACT   Social Anxiety Doesn ’ t Get The Attention It Deserves 
IMPACT   The Battle To Save Our Dying Soil 
IMPACT   A New Farming Technique Using Drastically Less Water Is Catching On 
IMPACT   6 Incredible Photos That Show The World We Need To Protect 
IMPACT   Our Snack Addiction Is Killing Orangutans 


## Evaluate a model accuracy
The result will be saved in list **prediction** and text file **prediction.log**.

In [4]:

def save_prediction(prediction, file_name):
    fo = open(file_name, "w+")
    for item in prediction:
        fo. write(str(item)+"\n")
    fo.close()
MODEL_FILE = 'news_category-100000.meta'
ckpt_dir = './ckpt_save/'
prediction_1 = rp.run_cnn_model(ckpt_dir,MODEL_FILE)
#print(len(prediction_1))

word_matrix_file = "naive_bayes_word_matrix.csv"
prediction_2 = rp.run_naive_bayes_model(word_matrix_file)
#print(len(prediction_2))

KeyboardInterrupt: 

In [ ]:
from NewsCategoryData import NewsCategoryTrainTestSet 
word_matrix_file = "naive_bayes_word_matrix_ver1.csv"
word_matrix, word_list = rp.read_naive_bayes_word_vector(word_matrix_file)

from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

glove_file = datapath('c:/Notebook/GloVe/glove.6B.100d.txt')
word2vec_glove_file = get_tmpfile("glove.6B.100d.word2vec.txt")

model = KeyedVectors.load_word2vec_format(word2vec_glove_file)

def run_combined_model(data, ckpt_dir, graph_file, word_matrix, word_list):
    graph = tf.Graph()
    with graph.as_default():
        saver = tf.train.import_meta_graph(ckpt_dir + graph_file)
        #import tensor variables
        x_rnn = graph.get_tensor_by_name("input/x_rnn:0")
        x_naive_bayes = graph.get_tensor_by_name("input/x_naive_bayes:0")
        x_length = graph.get_tensor_by_name("input/x_length:0")
        prediction = graph.get_tensor_by_name("Prediction/prediction:0")
    
        with tf.Session() as sess:
            model_file=tf.train.latest_checkpoint(ckpt_dir)
            saver.restore(sess,model_file)
            batch_data_vec, data_length = ncd.word2vector(data, model, 100)
            naive_bayes_input = rp.naive_bayes_model(batch_data, word_matrix, word_list)
            y= sess.run([prediction],
                        feed_dict={x_rnn:batch_data_vec, 
                                   x_naive_bayes:naive_bayes_input,
                                   x_length:data_length})

    return y[0]

data_path = "news_cat_train_test_data.csv"
ckpt_dir = "./ckpt_save_combine_model/"
graph_file = "news_category-100000.meta"

train_test_data = NewsCategoryTrainTestSet(filepath=data_path, batch_size=100)
prediction_list = []
for i in range(int(train_test_data.test_size/100)+1):
    batch_data, _ = train_test_data.batch_test_set()
    prediction = run_combined_model(batch_data, ckpt_dir, graph_file, word_matrix, word_list)
    #print(prediction)
    prediction_list = prediction_list + [i  for i in prediction]


In [6]:
def cal_accuracy(data, prediction):
    accuracy = 0
    for i  in range(data.max_recorder):
        if prediction[i] == data.label[i]:
            accuracy += 1
    return accuracy/data.max_recorder
print(cal_accuracy(data,prediction_1)*100)
print(cal_accuracy(data,prediction_2)*100)

52.42697177453485
62.20356788998591


## Check the accuracy of each label.

In [5]:
def check_accuracy_by_label(prediction, label):
    count_correct_by_category = [0 for item in LABEL_DIC]
    count_total_by_category = [0 for item in LABEL_DIC]

    for i in range(len(label)):
        count_total_by_category[label[i]] += 1
        if label[i] == prediction[i]:
            count_correct_by_category[label[i]] += 1
    accuracy = []
    #print(count_correct_by_category)
    #print(count_total_by_category)
    for i in range(len(count_correct_by_category)):
        accuracy.append(count_correct_by_category[i]/count_total_by_category[i])

    TRAIN_SAMPLE_LEVEL = [5000,2000,1000,500,200]
    train_sample_level = [(float('inf'), TRAIN_SAMPLE_LEVEL[0])]
    for i in range(len(TRAIN_SAMPLE_LEVEL)-1):
        train_sample_level.append((TRAIN_SAMPLE_LEVEL[i],TRAIN_SAMPLE_LEVEL[i+1]))
    train_sample_level.append((TRAIN_SAMPLE_LEVEL[len(TRAIN_SAMPLE_LEVEL)-1],0))
    print(train_sample_level)

    for level in train_sample_level:
        for i in range(len(count_correct_by_category)):
            if count_total_by_category[i] <= level[0] and count_total_by_category[i] > level[1]:
                print("%s:\t  %.2f %% \t %d \t%.2f %%"%(LABEL_LIST[i],accuracy[i]*100, count_total_by_category[i],count_total_by_category[i]/len(label)*100))
        print("*"*20)
#check_accuracy_by_label(prediction_1,data.label)
#check_accuracy_by_label(prediction_2,data.label)
check_accuracy_by_label(prediction_list, train_test_data.test_label)

NameError: name 'prediction_list' is not defined

In [9]:
import Run_Prediction as rp
import numpy as np
np.set_printoptions(precision=4,suppress =False)
word_matrix_file = "naive_bayes_word_matrix.csv"
word_matrix, word_list = rp.read_naive_bayes_word_vector(word_matrix_file)
title_list = [['Hillary', 'Set', 'To', 'Move', 'Past', 'Prelims', 'With', 'Roosevelt', 'Island', 'Address', '(', 'Are', 'the', 'Clintons', 'Cynics', 'or', 'Realists', '?', ')']]
for title in title_list:
    naive_bayes_input = rp.naive_bayes_model(title, word_matrix, word_list)
    print(naive_bayes_input)
    print(naive_bayes_input.argmax())
    print(naive_bayes_input.shape)
    print("-"*30)
 

Load the naive bayes word vectors. It takes a few minutes.
[[2.6427e-05 3.1536e-04 0.0000e+00 1.4195e-04 3.4879e-03 1.7384e-04
  2.6860e-04 1.5240e-03 8.8841e-04 2.8942e-04 7.1722e-05 1.4702e-04
  0.0000e+00 9.5298e-04 4.5360e-05 1.2278e-04 0.0000e+00 5.3821e-04
  2.0672e-04 1.7493e-04 2.2368e-04 5.7618e-04 3.7574e-04 1.1148e-04
  1.4270e-04 6.2869e-05 4.9418e-05 1.2218e-04 4.0393e-05 3.0862e-04
  0.0000e+00 0.0000e+00 1.1541e-05 0.0000e+00 2.3623e-04 0.0000e+00
  5.2102e-05 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [5.2854e-04 6.8508e-04 2.8423e-04 5.6779e-05 2.6386e-04 2.4338e-04
  3.2615e-04 8.3126e-05 1.0452e-04 2.3430e-04 1.9724e-04 2.1237e-04
  2.1262e-04 1.9717e-04 3.1752e-04 1.2278e-04 1.6947e-04 2.3066e-04
  2.0672e-04 0.0000e+00 1.2427e-04 2.5608e-04 4.1748e-05 1.4864e-04
  4.7567e-05 1.1002e-04 2.4709e-04 6.1091e-05 4.4432e-04 7.7155e-05
  2.0138e-04 1.4066e-04 1.3849e-04 3.9460e-04 1.7106e-04 1.1379e-04
  1.8236e-04 1.2391e-04 1.1193e-04 1.3905e-04 2.6187e-04]
 [1.7125e

## Generate Trainset and Testset 

For each category, 90% of recordes will be assigned as trainset and 10% will be assigned as test set.

In [11]:
# First step to get the count of recorders for each category.
import csv
def count_by_category(labels,class_mum):
    count_by_cat_list = [0 for i in range(class_mum)]
    for i in labels:
        count_by_cat_list[i] += 1
    return count_by_cat_list

lst_count_by_category = count_by_category(data.label,len(LABEL_DIC))

print(lst_count_by_category)

total_data = 0
for num in lst_count_by_category:
    total_data += num
print(total_data)

#Generate trainset and testset
train_data = []
train_label = []
test_data = []
test_label = []
count_train_by_category = [0 for i in range(len(LABEL_DIC))]

for i in range(data.max_recorder):
    if count_train_by_category[data.label[i]] <= lst_count_by_category[data.label[i]]*0.9:
        train_data.append(data.data[i]) 
        train_label.append(data.label[i])
        count_train_by_category[data.label[i]] += 1
    else:
        test_data.append(data.data[i]) 
        test_label.append(data.label[i])
        
# Now save the trainset and testset to the csv file.
# For each recorder, there's label to mark whether it is belong to train or test set. 
fo = open("news_cat_train_test_data.csv", "w+", encoding="utf-8")
writer = csv.writer(fo)
writer.writerow(['title','label','train_or_test'])
for i in range(len(train_data)):
    writer.writerow([train_data[i],train_label[i],'train'])
for i in range(len(test_data)):
    writer.writerow([test_data[i],test_label[i],'test'])
fo.close()

[3405, 16058, 2177, 3459, 32738, 2670, 4528, 3490, 5175, 6313, 4884, 5936, 9887, 2814, 2082, 2555, 2178, 1129, 1004, 1144, 3955, 1339, 2254, 2622, 2096, 6694, 3664, 1398, 2578, 1401, 1509, 17827, 8677, 4195, 9649, 3426, 3651, 6226, 1707, 1323, 1030]
200847


## Below is to test a class to read news category data which is seperated by train and test set

In [1]:
from NewsCategoryData import NewsCategoryTrainTestSet

dataset = NewsCategoryTrainTestSet(filepath = "news_cat_train_test_data.csv")
for i in range(10000):
    train_data, train_label = dataset.batch_test_set()
    if len(train_data) != 100 or len(train_label) != 100:
        print(len(train_data),len(train_label))
print(train_data[0], train_label[0])   

for i in range(10000):
    test_data, test_label = dataset.batch_test_set()
    if len(test_data) != 100 or len(test_label) != 100:
        print(len(test_data),len(test_label))
print(test_data[0], test_label[0])      

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


200847
['The', 'Buffett', 'Rule', 'or', 'the', 'Lincome', 'Tax', '?'] 38
['Setting', 'Sail', 'On', 'A', 'Schooner', 'For', 'A', 'Knitting', 'Vacation'] 12


## Below code will run the combined model to give category prediciton

In [9]:
import Run_Prediction as rp

ckpt_dir = './ckpt_save_8/'
model_file='news_category-200000.meta'

word_matrix_file = "naive_bayes_word_matrix.csv"
word_matrix, word_list = rp.read_naive_bayes_word_vector(word_matrix_file)

ckpt_dir_combine = './ckpt_save/'
graph_file = 'combined_new_category-50000.meta'

SyntaxError: invalid syntax (<ipython-input-9-9374289aa1b2>, line 5)

In [10]:
title_list= [["aust","addresses","un","security","council","over", "iraq"],
             ["funds","allocated","for","youth","at","risk"],
             ["police", "defend","aboriginal","tent","embassy","raid"]]
for title in title_list:
    cnn_logits = rp.one_cnn_model(title,ckpt_dir, model_file)
    naive_bayes_input = rp.one_naive_bayes_model(title, word_matrix, word_list)
    prediction = rp.one_combined_model(cnn_logits,naive_bayes_input,ckpt_dir_combine,graph_file)
    print(prediction[0][0])

Load graph.
Load model.
INFO:tensorflow:Restoring parameters from ./ckpt_save_8/news_category-200000
./ckpt_save/combined_new_category-50000
INFO:tensorflow:Restoring parameters from ./ckpt_save/combined_new_category-50000
Restore model.
2
Load graph.
Load model.
INFO:tensorflow:Restoring parameters from ./ckpt_save_8/news_category-200000
./ckpt_save/combined_new_category-50000
INFO:tensorflow:Restoring parameters from ./ckpt_save/combined_new_category-50000
Restore model.
4
Load graph.
Load model.
INFO:tensorflow:Restoring parameters from ./ckpt_save_8/news_category-200000
./ckpt_save/combined_new_category-50000
INFO:tensorflow:Restoring parameters from ./ckpt_save/combined_new_category-50000
Restore model.
0


In [6]:
# Find blank titles
import numpy as np
print(np.__version__)

1.18.1
